# Description

According to the settings specified below, this notebook:
 1. reads all the data from one source (GTEx, recount2, etc) according to the gene selection method (`GENE_SELECTION_STRATEGY`),
 2. runs a quick performance test using the correlation coefficient specified (`CORRELATION_METHOD`), and
 3. computes the correlation matrix across all the genes using the correlation coefficient specified.

# Modules

In [ ]:
import pandas as pd
from tqdm import tqdm

from clustermatch import conf
from clustermatch.corr import spearman

# Settings

In [ ]:
# we don't have gene subsets for recount2
# GENE_SELECTION_STRATEGY = "var_raw"

In [ ]:
CORRELATION_METHOD = spearman

method_name = CORRELATION_METHOD.__name__
display(method_name)

In [ ]:
PERFORMANCE_TEST_N_TOP_GENES = 500

# Paths

In [ ]:
INPUT_FILE = conf.RECOUNT2["DATA_FILE"]
display(INPUT_FILE)

assert INPUT_FILE.exists()

In [ ]:
OUTPUT_DIR = conf.RECOUNT2["SIMILARITY_MATRICES_DIR"]
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

# Data loading

In [ ]:
data = pd.read_pickle(INPUT_FILE)

In [ ]:
data.shape

In [ ]:
data.head()

# Compute similarity

## Performance test

In [ ]:
# select a subset of the genes
test_data = data.sample(n=PERFORMANCE_TEST_N_TOP_GENES, random_state=0)

In [ ]:
test_data.shape

In [ ]:
test_data.head()

This is a quick performance test of the correlation measure. The following line (`_tmp = ...`) is the setup code, which is needed in case the correlation method was optimized using `numba` and needs to be compiled before performing the test.

In [ ]:
_tmp = CORRELATION_METHOD(test_data.iloc[:3])

display(_tmp.shape)
display(_tmp)

In [ ]:
%timeit CORRELATION_METHOD(test_data)

## Run

In [ ]:
# compute correlations
data_corrs = CORRELATION_METHOD(data)

In [ ]:
output_filename = OUTPUT_DIR / f"{INPUT_FILE.stem}-{method_name}.pkl"
display(output_filename)

In [ ]:
# save
data_corrs.to_pickle(output_filename)